# Ключи, индексы и ограничения

В PostgreSQL можно наложить ограничения на колонки. Например, сделать колонки уникальным и ограничить его потенциальными значениями.

In [1]:
import psycopg2
from sql import exec_sql, sql_query
conn = psycopg2.connect(dbname='dvdrental', user='postgres',
                        password='postgres', host='localhost', port=5432)
cur = conn.cursor()

In [9]:
# Давайте создадим новую таблицу, где будет пол клиента и его id.
q = """
CREATE TABLE customer_sex
(
    id serial primary key,
    sex VARCHAR(2)
);
"""
exec_sql(q, conn)

In [8]:
# Если таблица уже существует, то ее можно удалить
# exec_sql("""
# DROP table customer_sex
# """, conn)

In [10]:
query = """
INSERT INTO  customer_sex(sex)
VALUES ('F'), ('M'), ('F'), ('R')
"""
exec_sql(query, conn)

In [11]:
sql_query("""SELECT * FROM customer_sex""")

,id,sex
0,1,F
1,2,M
2,3,F
3,4,R


# Ограничения

Как видим, одно из значений пола совсем нереалистичное. Давайте добавим специальный индекс к столбцу пола.

In [12]:
query = """
ALTER TABLE customer_sex
ADD CONSTRAINT unique_sex CHECK ( sex = 'F' OR sex = 'M' )
"""
exec_sql(query, conn)

check constraint "unique_sex" of relation "customer_sex" is violated by some row



In [13]:
q = """
DELETE FROM customer_sex
WHERE sex = 'R';

ALTER TABLE customer_sex
ADD CONSTRAINT unique_sex CHECK ( sex = 'F' OR sex = 'M' );
"""
exec_sql(q, conn)

In [14]:
sql_query("""SELECT * FROM customer_sex""")

,id,sex
0,1,F
1,2,M
2,3,F


In [15]:
exec_sql("""INSERT INTO customer_sex(sex)
VALUES ('R') """, conn)

new row for relation "customer_sex" violates check constraint "unique_sex"
DETAIL:  Failing row contains (5, R).



In [18]:
# Теперь создадим таблицу с электронными почтами клиентов
exec_sql("""
CREATE TABLE customer_email
(
id serial,
email VARCHAR(50)
)
""", conn)

In [17]:
# exec_sql("""
# DROP table customer_email
# """, conn)

В реальности у каждого юзера должна быть одна электронная почта. Давайте наложим уникальное ограничение
на столбец email.

In [19]:
exec_sql("""
ALTER TABLE customer_email
ADD CONSTRAINT unique_email UNIQUE(email)
""", conn)

In [20]:
exec_sql("""
INSERT INTO customer_email(email)
VALUES ('1@mail.ru'), ('2@gmail.com'), ('2@gmail.com')
""", conn)

duplicate key value violates unique constraint "unique_email"
DETAIL:  Key (email)=(2@gmail.com) already exists.



PostgreSQL выдал ошибку об уникальности столбца
Ниже посмотрим на список всех ограничений базы dvdrental.

In [22]:
sql_query("""
select pgc.conname as constraint_name,
       ccu.table_schema as table_schema,
       ccu.table_name,
       ccu.column_name
from pg_constraint pgc
join pg_namespace nsp on nsp.oid = pgc.connamespace
join pg_class  cls on pgc.conrelid = cls.oid
left join information_schema.constraint_column_usage ccu
          on pgc.conname = ccu.constraint_name
          and nsp.nspname = ccu.constraint_schema
order by pgc.conname;
""")

,constraint_name,table_schema,table_name,column_name
0,actor_pkey,public,actor,actor_id
1,address_pkey,public,address,address_id
2,category_pkey,public,category,category_id
3,city_pkey,public,city,city_id
4,country_pkey,public,country,country_id
...,...,...,...,...
205,store_address_id_fkey,public,address,address_id
206,store_manager_staff_id_fkey,public,staff,staff_id
207,store_pkey,public,store,store_id
208,unique_email,public,customer_email,email


# Индексы

Индексы нужны для того, чтобы увеличить скорость выполнения запросов.
Однако, это увеличивает место за жестком диске, которое расходуется на создание этих индексов.

In [23]:
# Список всех индексов базы dvdrental
q = """
SELECT
    tablename,
    indexname,
    indexdef
FROM
    pg_indexes
WHERE
    schemaname = 'public'
ORDER BY
    tablename,
    indexname;
"""
sql_query(q)

,tablename,indexname,indexdef
0,actor,actor_pkey,CREATE UNIQUE INDEX actor_pkey ON public.actor...
1,actor,idx_actor_last_name,CREATE INDEX idx_actor_last_name ON public.act...
2,address,address_pkey,CREATE UNIQUE INDEX address_pkey ON public.add...
3,address,idx_fk_city_id,CREATE INDEX idx_fk_city_id ON public.address ...
4,category,category_pkey,CREATE UNIQUE INDEX category_pkey ON public.ca...
5,city,city_pkey,CREATE UNIQUE INDEX city_pkey ON public.city U...
6,city,idx_fk_country_id,CREATE INDEX idx_fk_country_id ON public.city ...
7,country,country_pkey,CREATE UNIQUE INDEX country_pkey ON public.cou...
8,customer,customer_pkey,CREATE UNIQUE INDEX customer_pkey ON public.cu...
9,customer,idx_fk_address_id,CREATE INDEX idx_fk_address_id ON public.custo...


# Ключи

Реляционность базы обеспечивают первичный (primary) и внешний (foreign) ключи.
Они дают пользователю представления о том, как таблицы связаны друг с другом.
Также они защищают от удаления строк, которые связаны с другой таблицей при помощи внешних ключей.
Давайте посмотрим на список индексов, ключей и ограничений базы `dvdrental`.

In [24]:
from p_tqdm import p_map
import pandas as pd
list_all_tables = sql_query("""
SELECT tablename
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
    schemaname != 'information_schema';
""")['tablename'].tolist()

def get_keys_table(table_name):

    data = sql_query(f"""
    select  constraint_name, table_name, column_name
    from information_schema.key_column_usage
    where constraint_catalog=current_catalog and table_name='{table_name}'
    and position_in_unique_constraint notnull;
    """)
    return data
all_keys = list(p_map(get_keys_table, list_all_tables))
all_keys = pd.concat(all_keys)

  0%|          | 0/17 [00:00<?, ?it/s]